In [ ]:
from __future__ import print_function
import requests
import sys
import time
import urllib
from urllib.request import urlopen
import urllib.request
#import simplejson
import json
import shutil
import glob
#from PIL import Image
from pathlib import Path
#import cv2
#import numpy as np
import io
import xml.etree.ElementTree as ET
import base64
import pandas as pd
import tensorflow as tf
from datetime import datetime
import torch
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np
import xgboost as xgb
import math
from sklearn.metrics import r2_score
import seaborn as sns
from sklearn.metrics import mean_squared_log_error
from sklearn import preprocessing
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix

# Read data

In [2]:
from ast import literal_eval 
processed_df = pd.read_csv('')

In [ ]:
print(processed_df['fwd_citation_class'])

In [ ]:
y_df = pd.DataFrame(processed_df, columns =['fwd_citation_class'])
sns.displot(y_df, x = "fwd_citation_class")

In [ ]:
print(processed_df.columns)

In [6]:
processed_df = processed_df[processed_df.ipc_tech_field_one_hot != "0"]


In [ ]:
Counter(processed_df['fwd_citation_class'])

In [8]:
#processed_df["concat"] = processed_df["concat"].apply(lambda x: str(x.split(',')))
processed_df["doc_techfield"] = processed_df["doc_techfield"].apply(lambda x: literal_eval(x))
processed_df["doc_techfield"] = processed_df["doc_techfield"].apply(lambda x: np.array(x))


In [141]:
processed_df["doc_vector"] = processed_df["doc_vector"].apply(lambda x: literal_eval(x))
processed_df["doc_vector"] = processed_df["doc_vector"].apply(lambda x: np.array(x))

In [ ]:
print(processed_df["doc_techfield"])

In [10]:
shuffled_df = processed_df.sample(frac=1).reset_index(drop=True)

In [ ]:
X = shuffled_df["doc_techfield"].to_numpy()
X = np.vstack(X)
print(X.shape)

In [12]:
y = shuffled_df["fwd_citation_class"].values

In [ ]:
print(Counter(y))

In [28]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.25, random_state=42, stratify=y_)

In [ ]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_test))

## TSNE

In [ ]:
#umap dim reduction

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=4)
embedding = tsne.fit_transform(X)
#X_train, X_test, y_train, y_test = train_test_split(embedding, y, test_size=0.3, random_state=42)

In [ ]:
X_, X_test, y_, y_test = train_test_split(embedding, y, test_size=0.1, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.25, random_state=42, stratify=y_)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.scatter(embedding, y, s=10, color='blue')
plt.title('2D Plot of y vs. X', fontsize=16)
plt.xlabel('X', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the data points
ax.scatter(embedding[:, 0], embedding[:, 1], y_train, c=y_train, cmap='viridis')

# Set labels and title
ax.set_xlabel('Vector Dimension 1', fontsize=12)
ax.set_ylabel('Vector Dimension 2', fontsize=12)
ax.set_zlabel('Dependent Variable', fontsize=12)
ax.set_title('3D Plot with Vector Data and Dependent Variable', fontsize=14)

plt.show()

In [ ]:
print(X_train.shape)
print(X_train[0].shape)
X_train = np.expand_dims(X_train, axis=1)
print(X_train.shape)

# PCA

In [ ]:
from sklearn.decomposition import PCA as sklearnPCA

sklearn_pca = sklearnPCA(n_components=100)
embedding = sklearn_pca.fit_transform(X)
print(sklearn_pca.explained_variance_ratio_)

In [ ]:
X_, X_test, y_, y_test = train_test_split(embedding, y, test_size=0.1, random_state=42, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_, y_, test_size=0.25, random_state=42, stratify=y_)

# Random Forest

base - n_estimators = 100, max_depth = 2, acc = 0.56


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

#hyperparameters
n_estimators = 100
max_depth = 2

clf = RandomForestClassifier(n_estimators= n_estimators, max_depth= max_depth, random_state=42)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(predictions)

In [ ]:
print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

# XGBoost

base - 0.66

In [ ]:
# encoding y

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [ ]:
#xgboost
learning_rate = 0.05
model = xgb.XGBClassifier()
'''
    objective='multi:softprob',
    num_class=7,      
    max_depth=max_depth,       
    learning_rate=learning_rate,
    #subsample=subsample,        
    #colsample_bytree=colsample, 
    n_estimators=n_estimators
)
'''
model.fit(X_train, y_train)
predictions = model.predict(X_test)
# inverse transform predictions
predictions = le.inverse_transform(predictions)

print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

In [ ]:
print(Counter(y_test))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats as stats

#hyperparameter distributions
param_dist = {
    'max_depth': stats.randint(3, 10),
    'eta': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}

# XGBoost model object
xgb_model = xgb.XGBClassifier()
#grid_search = GridSearchCV(xgb_model, param_grid, cv=5, scoring='accuracy')
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=10, cv=5)


# Fit the GridSearchCV object to the training data
random_search.fit(X_train, y_train)

# Print the best set of hyperparameters and the corresponding score
print("Best set of hyperparameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

model = xgb.XGBClassifier(learning_rate = random_search.best_params_["eta"],
                        max_depth  = random_search.best_params_["max_depth"],
                        subsample = random_search.best_params_["subsample"],
                        n_estimators = random_search.best_params_["n_estimators"])
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# inverse transform predictions
predictions = le.inverse_transform(predictions)

print(classification_report(y_test, predictions))
print(confusion_matrix(y_test, predictions))

In [ ]:
# Plot the results
idx = [ i for i in range(10381)]
plt.plot(idx, y_test, color='blue', label='Actual')
plt.plot(idx, predictions, color='red', label='Predicted')
plt.title('XG Boost Regression')
plt.xlabel('Input feature')
plt.ylabel('Target variable')
plt.legend()
plt.show()

In [ ]:
print(np.mean(y_test))
print(np.std(y_test))
print(np.var(y_test))

print("\n\n")

print(np.mean(predictions))
print(np.std(predictions))
print(np.var(predictions))

### SVM

In [ ]:
from sklearn.svm import SVR
import matplotlib.pyplot as plt

svm_regressor = SVR(kernel='poly', C=10, gamma='scale')  #RBF kernel for non-linear regression
svm_regressor.fit(X_train, y_train)

predictions = svm_regressor.predict(X_test)

#Calculate Mean Squared Error (MSE) as a performance metric
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

In [ ]:
# Calculate R-squared value
r2 = r2_score(y_test, predictions)
print("R-squared svm regressor", r2)

print("Root Mean Squared Error:", mse**0.5)

print(np.mean(y_test))
print(np.std(y_test))
print(np.var(y_test))

print("\n\n")

print(np.mean(predictions))
print(np.std(predictions))
print(np.var(predictions))

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predictions)
print(mae)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
import matplotlib.pyplot as plt

param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVR(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

print(grid.best_estimator_)

predictions = grid.predict(X_test)
#Calculate Mean Squared Error (MSE) as a performance metric
mse = mean_squared_error(y_test, predictions)
print("Root Mean Squared Error:", mse**0.5)

In [ ]:
from sklearn.metrics import r2_score

# Calculate R-squared value
r2 = r2_score(y_test, predictions)
print("R-squared svm regressor", r2)

# Plot the results
idx = [ i for i in range(31142)]
plt.scatter(idx, y_test, color='blue', label='Actual')
plt.scatter(idx, predictions, color='red', label='Predicted')
plt.title('Support Vector Machine (SVM) Regression')
plt.xlabel('Input feature')
plt.ylabel('Target variable')
plt.legend()
plt.show()

In [ ]:
#Artificial Neural Network
from keras.optimizers import Adam
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

train = X_train
train= np.reshape(train,(train.shape[0], 1, train.shape[1]))
val=X_val
val= np.reshape(X_val,(X_val.shape[0], 1, X_val.shape[1]))

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
target = to_categorical(np.array(y_train))
val_target = to_categorical(np.array(y_val), num_classes=5)

'''
ohe = OneHotEncoder()
ohe.fit(y_train.reshape(-1, 1))
target = ohe.transform(y_train.reshape(-1, 1)).toarray()

ohe.fit(y_test.reshape(-1, 1))
val_target = ohe.transform(y_val.reshape(-1, 1)).toarray()

ohe.fit(y_test.reshape(-1, 1))
test_target = ohe.transform(y_test.reshape(-1, 1)).toarray()
'''


test = X_test
test= np.reshape(test,(test.shape[0], 1, test.shape[1]))


NN_model = tf.keras.Sequential()
#NN_model.add(tf.keras.layers.Dense(64, kernel_initializer='normal',input_dim = train.shape[1], activation='relu'))
#NN_model.add(tf.keras.layers.Dropout(0.25))
#NN_model.add(tf.keras.layers.Dense(256, kernel_initializer='normal', input_dim = train.shape[1], activation='relu'))
#NN_model.add(tf.keras.layers.Dropout(0.5, input_dim = train.shape[1]))
#NN_model.add(tf.keras.layers.Dense(256, kernel_initializer='normal', input_dim = train.shape[1], activation='relu'))
NN_model.add(tf.keras.layers.GRU(256, return_sequences=True))#, recurrent_dropout=0.25, dropout=0.25))
NN_model.add(tf.keras.layers.Dropout(0.25))
NN_model.add(tf.keras.layers.GRU(128, return_sequences=True))
#NN_model.add(tf.keras.layers.Dropout(0.5))
NN_model.add(tf.keras.layers.GRU(64, return_sequences=True))
#NN_model.add(tf.keras.layers.Dense(64, kernel_initializer='normal', activation='relu'))
#NN_model.add(tf.keras.layers.Dropout(0.25))

NN_model.add(tf.keras.layers.GRU(32))
#NN_model.add(tf.keras.layers.Dense(32, kernel_initializer='normal',activation='relu'))
#NN_model.add(tf.keras.layers.Dropout(0.25))
NN_model.add(tf.keras.layers.Dense(8, kernel_initializer='normal',activation='relu'))
NN_model.add(tf.keras.layers.Dense(5, kernel_initializer='normal',activation='softmax'))
NN_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy'])
#print(NN_model.summary())

checkpoint_name = 'doc-tech-field-concat-GRU-256-0.25-128-64-32-dense-8-5-cce-{epoch:03d}--{val_loss:.5f}.hdf5'
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint, es]
history = NN_model.fit(train, target, epochs=1000, batch_size=32, callbacks=callbacks_list, validation_data=(val, val_target))
print(history)
print(NN_model)
predictions = NN_model.predict(test)

In [ ]:
print(history.history.keys())

In [ ]:
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()

In [ ]:
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='val')
pyplot.legend()
pyplot.show()

In [ ]:
# Plot the results
idx = [ i for i in range(31142)]
plt.scatter(idx, y_test, color='blue', label='Actual')
plt.scatter(idx, predictions, color='red', label='Predicted')
plt.title('NN')
plt.xlabel('Input feature')
plt.ylabel('Target variable')
plt.legend()
plt.show()

In [ ]:
plt.plot(y_test[:100], color='blue', linestyle = 'dotted')
plt.plot(predictions[:100], color='red', linestyle = 'dotted')
plt.show()